<div style="text-align:center;font-size:22pt; font-weight:bold;color:white;border:solid black 1.5pt;background-color:#1e7263;">
    Understanding Model History Object: Classification Task
</div>

In [1]:
# ======================================================================= #
# Course: Deep Learning Complete Course (CS-501)
# Author: Dr. Saad Laouadi
# Institution: Quant Coding Versity Academy
# Date: December 25, 2024
#
# ==========================================================
# Lesson: Understanding Model History Object in Keras
#         Analyzing and Visualizing Training Progress
# ==========================================================
# ## Learning Objectives
# This guide will enable you to:
# 1. Access and interpret the model.fit() history object
# 2. Extract and analyze training metrics over epochs
# 3. Visualize training and validation metrics
# 4. Identify optimal training epochs and model performance
# 5. Detect overfitting through history analysis
# =======================================================================
#          Copyright © Dr. Saad Laouadi 2024
# =======================================================================

In [2]:
# ==================================================== #
#        Load Required Libraries
# ==================================================== #

import os  
import shutil
from datetime import datetime

# Disable Metal API Validation
os.environ["METAL_DEVICE_WRAPPER_TYPE"] = "0"  


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# import tensorflow
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Set styling for better visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("="*72)

%reload_ext watermark
%watermark -a "Dr. Saad Laouadi" -u -d -m

print("="*72)
print("Imported Packages and Their Versions:")
print("="*72)

%watermark -iv
print("="*72)

# Global Config
RANDOM_STATE = 101

Author: Dr. Saad Laouadi

Last updated: 2025-01-04

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Imported Packages and Their Versions:
numpy     : 1.26.4
keras     : 3.6.0
seaborn   : 0.13.2
matplotlib: 3.9.2
pandas    : 2.2.2
sklearn   : 1.5.1
tensorflow: 2.16.2



In [3]:
def cleanup_directory(directory_path):
    """
    Deletes the specified directory and all its contents.

    Args:
        directory_path (str): Path to the directory to delete.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        shutil.rmtree(directory_path)
        print(f"Directory '{os.path.basename(directory_path)}' deleted successfully.")
    else:
        print(f"Directory '{os.path.basename(directory_path)}' does not exist or is not a directory.")

### ML Model Raodmap
1. Read the data
2. Explore the data

3. Processing
    - Numerical features (scaling)
    
4. Splitting the data

In [6]:
# ==================================================== #
#        Implementing EarlyStopping  
#        Callback with Synthetic data
# ==================================================== #

In [6]:
# Create synthetic dataset (same as before)
def create_synthetic_data(n_samples: int = 1000, random_state = 0):
    """
    Generate random data. 
    """
    X, y = make_classification(
        n_samples=n_samples,
        n_features=20,
        n_informative=15,
        n_redundant=5,
        random_state=random_state
    )
    
    y = tf.keras.utils.to_categorical(y)
    
    return train_test_split(X, y, test_size=0.2, random_state=random_state)

In [7]:
# Create a simple neural network model
def create_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape = input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [8]:
# Main training function with early stopping

# Generate synthetic data
X_train, X_test, y_train, y_test = create_synthetic_data(random_state=RANDOM_STATE)
print(X_train.shape)
print(y_train.shape)

(800, 20)
(800, 2)


In [9]:
# Create model
model = create_model((X_train.shape[1],))

print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,490 (13.63 KB)

 Trainable params: 3,490 (13.63 KB)

 Non-trainable params: 0 (0.00 B)

None


In [16]:
# Create checkpoint directory if it doesn't exist
checkpoint_dir = 'model_checkpoints'

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [12]:
print(os.getcwd())

/Users/saad/Trainings/DeepLearningCourse/lectures/03-05-building-models-with-keras-practical


In [18]:
# os.listdir()

In [13]:
# Define different types of EarlyStopping callbacks

# 1. Basic early stopping monitoring validation loss
early_stopping_basic = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1
)

# Train the model with early stopping callback
history = model.fit(
    X_train, y_train,
    epochs=100,           # Set a high number of epochs, early stopping will prevent overfitting
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        early_stopping_basic
    ],
    verbose=1
)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5926 - loss: 0.8206 - val_accuracy: 0.7437 - val_loss: 0.5276
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7910 - loss: 0.4286 - val_accuracy: 0.8250 - val_loss: 0.4115
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8866 - loss: 0.2863 - val_accuracy: 0.8687 - val_loss: 0.3430
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9214 - loss: 0.2478 - val_accuracy: 0.8938 - val_loss: 0.3146
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9419 - loss: 0.1979 - val_accuracy: 0.9000 - val_loss: 0.2931
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9564 - loss: 0.1593 - val_accuracy: 0.9125 - val_loss: 0.2693
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9535 - loss: 0.1449 - val_accuracy: 0.9000 - val_loss: 0.2651
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9661 - loss: 0.1283 - val_accuracy: 0.906

In [16]:
# 2. Early stopping with restoration of best weights
early_stopping_with_restore = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True
)
# Train the model with early stopping callback
history = model.fit(
    X_train, 
    y_train,
    batch_size=16, 
    epochs=150,
    validation_split=0.15,
    verbose=1, 
    callbacks=[early_stopping_with_restore]
)

Epoch 1/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0047 - val_accuracy: 0.9500 - val_loss: 0.2346
Epoch 2/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 0.9417 - val_loss: 0.2462
Epoch 3/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 0.9500 - val_loss: 0.2436
Epoch 4/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0032 - val_accuracy: 0.9417 - val_loss: 0.2479
Epoch 5/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0029 - val_accuracy: 0.9417 - val_loss: 0.2489
Epoch 6/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 0.9500 - val_loss: 0.2456
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.


In [20]:


# 3. Early stopping with minimum change threshold
early_stopping_min_delta = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.0001,  # Minimum change to qualify as an improvement
    verbose=1
)


# Train the model with early stopping callback
history = model.fit(
    X_train, y_train,
    epochs=100,  
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        early_stopping_min_delta
    ],
    verbose=1
)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.9625 - val_loss: 0.1961
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 9.2424e-04 - val_accuracy: 0.9625 - val_loss: 0.2019
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 9.7272e-04 - val_accuracy: 0.9625 - val_loss: 0.1989
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 9.0629e-04 - val_accuracy: 0.9625 - val_loss: 0.1994
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 8.6249e-04 - val_accuracy: 0.9625 - val_loss: 0.2012
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 6.7580e-04 - val_accuracy: 0.9625 - val_loss: 0.2005
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 9.4097e-04 - val_accuracy: 0.9625 - val_loss: 0.2006
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 7.0645

In [21]:
# 4. Early stopping monitoring validation accuracy
early_stopping_accuracy = EarlyStopping(
    monitor='val_accuracy',
    mode='max',  # For accuracy, we want to maximize
    patience=5,
    verbose=1
)

# Train the model with early stopping callback
history = model.fit(
    X_train, y_train,
    epochs=100,  # Set a high number of epochs, early stopping will prevent overfitting
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        early_stopping_accuracy
    ],
    verbose=1
)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 6.6657e-04 - val_accuracy: 0.9688 - val_loss: 0.2039
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 7.8253e-04 - val_accuracy: 0.9625 - val_loss: 0.2057
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 6.8594e-04 - val_accuracy: 0.9688 - val_loss: 0.2053
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 6.9659e-04 - val_accuracy: 0.9688 - val_loss: 0.2058
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 6.6478e-04 - val_accuracy: 0.9688 - val_loss: 0.2075
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 6.7669e-04 - val_accuracy: 0.9688 - val_loss: 0.2065
Epoch 6: early stopping


In [22]:
# Combine with Checkpoint for best practice
checkpoint_best = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'best_model_early_stopping.keras'),
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

# Train the model with callbacks
history = model.fit(
    X_train, y_train,
    epochs=100,  # Set a high number of epochs, early stopping will prevent overfitting
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        early_stopping_basic,
        early_stopping_with_restore,
        early_stopping_min_delta,
        early_stopping_accuracy,
        checkpoint_best
    ],
    verbose=1
)

Epoch 1/100
15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 5.4456e-04
Epoch 1: val_loss improved from inf to 0.20646, saving model to model_checkpoints/best_model_early_stopping.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 5.6477e-04 - val_accuracy: 0.9688 - val_loss: 0.2065
Epoch 2/100
15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 6.4927e-04
Epoch 2: val_loss did not improve from 0.20646
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 6.3812e-04 - val_accuracy: 0.9625 - val_loss: 0.2094
Epoch 3/100
14/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 7.7340e-04
Epoch 3: val_loss did not improve from 0.20646
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 7.2003e-04 - val_accuracy: 0.9688 - val_loss: 0.2077
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 6.0602e-04
Epoch 4: val_loss did not improve from 0.20646
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s

In [22]:
def demonstrate_early_stopping_scenarios(X_train, y_train, X_test, y_test):
    """
    Demonstrates different early stopping scenarios and their effects
    """
    # Create checkpoint directory
    checkpoint_dir = 'model_checkpoints'
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    # Scenario 1: Basic early stopping
    print("\nScenario 1: Basic Early Stopping")
    model1 = create_model((X_train.shape[1],))
    early_stopping1 = EarlyStopping(
        monitor='val_loss',
        patience=3,
        verbose=1
    )
    history1 = model1.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping1],
        verbose=1
    )

    # Scenario 2: Early stopping with baseline
    print("\nScenario 2: Early Stopping with Baseline Target")
    model2 = create_model((X_train.shape[1],))
    early_stopping2 = EarlyStopping(
        monitor='val_loss',
        baseline=0.3,  # Stop if we achieve this target loss
        patience=3,
        verbose=1
    )
    history2 = model2.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping2],
        verbose=1
    )

    # Scenario 3: Early stopping with best weights restoration
    print("\nScenario 3: Early Stopping with Best Weights Restoration")
    model3 = create_model((X_train.shape[1],))
    early_stopping3 = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )
    checkpoint3 = ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, 'scenario3_best_model.keras'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
    history3 = model3.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping3, checkpoint3],
        verbose=1
    )

    return {
        'basic': (model1, history1),
        'baseline': (model2, history2),
        'restore_best': (model3, history3)
    }


# Run different scenarios
results = demonstrate_early_stopping_scenarios(X_train, y_train, X_test, y_test)

# You can now compare the results
for scenario, (model, history) in results.items():
    val_loss = min(history.history['val_loss'])
    epochs_run = len(history.history['loss'])
    print(f"\nScenario: {scenario}")
    print(f"Best validation loss: {val_loss:.4f}")
    print(f"Training stopped after {epochs_run} epochs")


Scenario 1: Basic Early Stopping
Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6752 - loss: 0.7237 - val_accuracy: 0.8062 - val_loss: 0.4603
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8016 - loss: 0.4364 - val_accuracy: 0.8687 - val_loss: 0.3593
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8617 - loss: 0.3414 - val_accuracy: 0.8687 - val_loss: 0.3299
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8990 - loss: 0.2680 - val_accuracy: 0.8875 - val_loss: 0.3057
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9033 - loss: 0.2518 - val_accuracy: 0.8875 - val_loss: 0.2863
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9288 - loss: 0.2184 - val_accuracy: 0.8938 - val_loss: 0.2799
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9291 - loss: 0.1920 - val_accuracy: 0.9250 - val_loss: 0.2532
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9429 - loss

In [13]:
# Cleanup 
cleanup_directory(checkpoint_dir)

NameError: name 'checkpoint_dir' is not defined

## Key differences from the ModelCheckpoint example:

 - EarlyStopping is focused on preventing overfitting by monitoring training metrics
 - It includes parameters like patience and min_delta for fine-tuning stopping conditions
 - The restore_best_weights option can automatically restore the model to its best state
 - It can monitor different metrics (loss, accuracy) with different modes (min, max)
 - We set a higher number of epochs (100) since early stopping will prevent unnecessary training